Este projeto visa analisar as características e estilos de vida das pessoas para determinar suas probabilidades e potenciais de desenvolver um AVC no futuro.

In [30]:
!pip install scikit-learn==1.2.2

In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk

# **Carregando a Base de Dados**

https://www.kaggle.com/fedesoriano/stroke-prediction-dataset

In [32]:
data = pd.read_csv('healthcare-dataset-stroke-data.csv')
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


# **Pré-processamento dos Dados**

In [34]:
data.isna().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [35]:
data = data.dropna() #deletando os dados NaN
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [36]:
from sklearn.preprocessing import LabelEncoder #transformando as váriaveis com 2 respostas em números (0,1)
le = LabelEncoder()

for var in ['ever_married','Residence_type','gender','work_type','smoking_status']:
  data[var] = le.fit_transform(data[var])

data.head()

<ipython-input-36-9022af89de69>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[var] = le.fit_transform(data[var])
<ipython-input-36-9022af89de69>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[var] = le.fit_transform(data[var])
<ipython-input-36-9022af89de69>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,67.0,0,1,1,2,1,228.69,36.6,1,1
2,31112,1,80.0,0,1,1,2,0,105.92,32.5,2,1
3,60182,0,49.0,0,0,1,2,1,171.23,34.4,3,1
4,1665,0,79.0,1,0,1,3,0,174.12,24.0,2,1
5,56669,1,81.0,0,0,1,2,1,186.21,29.0,1,1


In [37]:
preditoras = data.iloc[:,0:11]
target = data.iloc[:, 11]

In [38]:
from imblearn.over_sampling import SMOTE

balanceador = SMOTE(random_state=9)

preditora_res, target_res = balanceador.fit_resample(preditoras, target)

In [39]:
target_res.value_counts()

stroke
1    4700
0    4700
Name: count, dtype: int64

In [40]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(preditora_res, target_res, test_size=0.2, random_state=9)

In [41]:
from sklearn.preprocessing import StandardScaler #escalando os dados para uma melhor otimização da rede neural

sc = StandardScaler()
x_train_normalizado = sc.fit_transform(x_train)
x_test_normalizado = sc.transform(x_test)

x_train

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
3991,71057,0,54.000000,0,0,1,2,0,70.190000,39.100000,3
7036,71845,0,75.406642,0,0,1,2,1,104.174646,24.956878,0
2826,26605,0,39.000000,0,0,1,2,0,102.510000,26.600000,3
7982,54405,1,79.984481,0,0,1,3,0,251.805014,30.524054,0
5182,66262,0,64.808038,0,0,0,3,0,67.578676,32.136217,1
...,...,...,...,...,...,...,...,...,...,...,...
6200,37705,0,77.063958,1,0,0,3,1,69.523556,29.029277,0
501,20751,0,26.000000,0,0,1,2,0,75.290000,22.600000,3
6782,16434,1,68.736208,0,0,0,2,1,115.282717,30.484655,1
4444,12812,0,53.000000,0,0,1,2,0,102.000000,32.400000,2


# **Aquitetura da Rede Neural**

In [46]:
  modelo = tf.keras.models.Sequential()
  modelo.add(Dense(units=6, activation='relu'))
  modelo.add(Dense(units=6, activation='relu'))
  modelo.add(Dense(units=1, activation='sigmoid'))
  modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
modelo.fit(x_train_normalizado, y_train, epochs=100, batch_size=32)

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5909 - loss: 0.6653
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7096 - loss: 0.5466
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7766 - loss: 0.4584
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7944 - loss: 0.4219
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8081 - loss: 0.3988
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8192 - loss: 0.3856
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8227 - loss: 0.3816
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8215 - loss: 0.3876
Epoch 9/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8299 - loss: 0.3718
Epoch 10/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8249 - loss: 0.3764
Epoch 11/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8349 - loss: 0.3575
Epoch 12/100
235/235 ━━━━━━━━━━━━━━━━━━━━

**Resultados:**

In [49]:
from sklearn.metrics import accuracy_score
y_pred = modelo.predict(x_test_normalizado)
y_pred = (y_pred > 0.5)
print(accuracy_score(y_test, y_pred))

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
0.8494680851063829


In [50]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [51]:
cm

array([[791, 159],
       [124, 806]])